# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [52]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sqlalchemy import create_engine

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [53]:
# load data from database
engine = create_engine('sqlite:///Messages.db')
df = pd.read_sql_table('Msg_table',engine)
X = df.message.values
Y = df[df.columns[4:]].values.astype('int64')

### 2. Write a tokenization function to process your text data

In [18]:
def tokenize(text):

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [19]:
pipeline = Pipeline([
        
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,random_state=3)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [21]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

y_pred = pipeline.predict(X_test)
f1_scores=np.zeros((36,1))
for i,col in enumerate(df.columns[4:]):
    print(f"{col} performance metrics")
    print(classification_report(y_test[:,i], y_pred[:,i]))
    f1_scores[i,]=f1_score(y_test[:,i], y_pred[:,i])
    print("##########################################################")
print(f"Average f1-score for this model is {np.mean(f1_scores)}")

related performance metrics
             precision    recall  f1-score   support

          0       0.68      0.62      0.65       879
          1       0.80      0.84      0.82      1631

avg / total       0.76      0.76      0.76      2510

##########################################################
request performance metrics
             precision    recall  f1-score   support

          0       0.82      0.92      0.87      1621
          1       0.81      0.63      0.71       889

avg / total       0.81      0.82      0.81      2510

##########################################################
offer performance metrics
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2505
          1       0.00      0.00      0.00         5

avg / total       1.00      1.00      1.00      2510

##########################################################
aid_related performance metrics
             precision    recall  f1-score   support

          

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [22]:
parameters = {
        
        'vect__max_df': ( 0.75,1.00),
        #'vect__max_features': (7500,10000),
        'clf__estimator__n_estimators': [100,200],}
        #'clf__estimator__min_samples_split': [3, 4],

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [23]:
cv.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__max_df': (0.75, 1.0), 'clf__estimator__n_estimators': [100, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [25]:
y_pred = cv.predict(X_test)

from sklearn.metrics import classification_report
f1_scores_cv=np.zeros((36,1))
for i,col in enumerate(df.columns[4:]):
    print(f"{col} performance metrics")
    print(classification_report(y_test[:,i], y_pred[:,i]))
    f1_scores_cv[i,]=f1_score(y_test[:,i], y_pred[:,i])
    print("##########################################################")
print(f"Average f1-score for this model is {np.mean(f1_scores_cv)}")

related performance metrics
             precision    recall  f1-score   support

          0       0.76      0.54      0.63       879
          1       0.79      0.91      0.84      1631

avg / total       0.78      0.78      0.77      2510

##########################################################
request performance metrics
             precision    recall  f1-score   support

          0       0.85      0.93      0.89      1621
          1       0.84      0.69      0.76       889

avg / total       0.84      0.84      0.84      2510

##########################################################
offer performance metrics
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2505
          1       0.00      0.00      0.00         5

avg / total       1.00      1.00      1.00      2510

##########################################################
aid_related performance metrics
             precision    recall  f1-score   support

          

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


#### As evident, this model is better as average F1-score improved from .194 to 0.211

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

##### Based on the above model, we first try to identify, which classes have lower F1-scores and number of data point available for training.

In [26]:
for i,col in enumerate(df.columns[4:]):
    
    print(f1_scores_cv[i],col,y_train[:,i].sum())

[ 0.8428246] related 5012
[ 0.7595561] request 2725
[ 0.] offer 5
[ 0.77111111] aid_related 2975
[ 0.06060606] medical_help 449
[ 0.06666667] medical_products 255
[ 0.04255319] search_and_rescue 161
[ 0.] security 107
[ 0.] military 34
[ 0.] child_alone 0
[ 0.67540984] water 591
[ 0.7827476] food 1155
[ 0.48199446] shelter 822
[ 0.] clothing 78
[ 0.] money 90
[ 0.] missing_people 65
[ 0.] refugees 129
[ 0.1] death 193
[ 0.02604167] other_aid 1085
[ 0.] infrastructure_related 235
[ 0.] transport 148
[ 0.17821782] buildings 290
[ 0.] electricity 55
[ 0.] tools 24
[ 0.] hospitals 45
[ 0.] shops 23
[ 0.] aid_centers 48
[ 0.] other_infrastructure 133
[ 0.64220183] weather_related 1097
[ 0.42553191] floods 211
[ 0.02898551] storm 211
[ 0.] fire 28
[ 0.76340694] earthquake 612
[ 0.] cold 49
[ 0.03703704] other_weather 141
[ 0.73622047] direct_report 2616


##### Class "storm" has very low F1-score (0.0289) even though the training samples available for it are comparable to compared to class "floods" with F1-score 0.426. Addressing tweets that seek information about imminent storm (cyclone, hurricane, etc) are important to prevent loss of life. After going through some of the tweets with "storm" as one of the classes, following associated words were identified.

* rain
* raining
* storm
* cyclone

We will implement an estimator class to extract word count of "storm" words. The code for the same is adapted from this repository https://github.com/rajatsharma369007/udacity-mentorship-repository/blob/master/nlp/custom_text_transformers.py

To further improve model on another classes too, we will apply a new algorithm from scikit learn called ExtraTreesClassifier

In [59]:
class StormWordCounter(BaseEstimator, TransformerMixin):
    '''
    Customized transformer for counting number of storm words in text.
    '''
    # Adding 'activate' parameter to activate the transformer or not:
    def __init__(self, activate = True):
        self.activate = activate

    # Defining fit method:
    def fit(self, X, y = None):
        return self

    # Defining transform method:
    def transform(self, X):
        '''
        It recieves an array of messages and counts the number of characters
        for each message.
        Input:
        X: array of text messages
        Output:
        elec_words_arr: array with the number of storm words for each
        message.
        '''
        # If activate parameter is set to True:
        if self.activate:
            st_words_count = list()
            st_list = ['rain','raining','storm','cyclone']#['shelter', 'home','house', 'housing', 'tent']
            # Counting shelter words:
            for text in X:
                # Creating empty list:
                st_words = 0
                tokens = word_tokenize(text.lower())
                for word in tokens:
                    if word in st_list:
                        st_words += 1
                st_words_count.append(st_words)
            # Transforming list into array:
            st_words_arr = np.array(st_words_count)
            st_words_arr = sh_words_arr.reshape((len(st_words_arr), 1))
            return st_words_arr

        # If activate parameter is set to False:
        else:
            pass

In [60]:
pipeline3 = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('storm_count',  StormWordCounter())
    ])),
    ('scale', StandardScaler(with_mean=False)),
    ('clf', MultiOutputClassifier(ExtraTreesClassifier()))
])

parameters = {
        
        'features__text_pipeline__vect__max_df': (0.5, 0.75),
        #'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'clf__estimator__n_estimators': [100,200],}
        #'clf__estimator__min_samples_split': [ 3, 4],}

cv_new = GridSearchCV(pipeline3, param_grid=parameters,cv=3)

In [61]:
#X_train, X_test, y_train, y_test = train_test_split(X, Y)
cv_new.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...
           oob_score=False, random_state=None, verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'features__text_pipeline__vect__max_df': (0.5, 0.75), 'clf__estimator__n_estimators': [100, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [62]:
y_pred = cv_new.predict(X_test)
f1_scores_nalg=np.zeros((36,1))
for i,col in enumerate(df.columns[4:]):
    print(f"{col} performance metrics")
    print(classification_report(y_test[:,i], y_pred[:,i]))
    f1_scores_nalg[i,]=f1_score(y_test[:,i], y_pred[:,i])
    print("##########################################################")
print(f"Average f1-score for this model is {np.mean(f1_scores_nalg)}")

related performance metrics
             precision    recall  f1-score   support

          0       0.78      0.55      0.64       879
          1       0.79      0.91      0.85      1631

avg / total       0.78      0.79      0.77      2510

##########################################################
request performance metrics
             precision    recall  f1-score   support

          0       0.85      0.92      0.88      1621
          1       0.83      0.70      0.76       889

avg / total       0.84      0.84      0.84      2510

##########################################################
offer performance metrics
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2505
          1       0.00      0.00      0.00         5

avg / total       1.00      1.00      1.00      2510

##########################################################
aid_related performance metrics
             precision    recall  f1-score   support

          

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [64]:
for i,col in enumerate(df.columns[4:]):
    
    print(f1_scores_nalg[i],col,y_train[:,i].sum())

[ 0.84724588] related 5012
[ 0.75883069] request 2725
[ 0.] offer 5
[ 0.75198188] aid_related 2975
[ 0.05882353] medical_help 449
[ 0.04444444] medical_products 255
[ 0.08163265] search_and_rescue 161
[ 0.] security 107
[ 0.] military 34
[ 0.] child_alone 0
[ 0.53090909] water 591
[ 0.69830508] food 1155
[ 0.4863388] shelter 822
[ 0.] clothing 78
[ 0.05555556] money 90
[ 0.] missing_people 65
[ 0.] refugees 129
[ 0.1] death 193
[ 0.04639175] other_aid 1085
[ 0.] infrastructure_related 235
[ 0.] transport 148
[ 0.26168224] buildings 290
[ 0.] electricity 55
[ 0.] tools 24
[ 0.] hospitals 45
[ 0.] shops 23
[ 0.] aid_centers 48
[ 0.] other_infrastructure 133
[ 0.61737523] weather_related 1097
[ 0.33707865] floods 211
[ 0.35714286] storm 211
[ 0.] fire 28
[ 0.67595819] earthquake 612
[ 0.] cold 49
[ 0.03703704] other_weather 141
[ 0.73649967] direct_report 2616


### 9. Export your model as a pickle file

In [65]:
import pickle
pickle.dump(cv_new.best_estimator_, open("bestmodel.pkl", 'wb'))

In [67]:
pickle.dump(cv_new, open("whole_gs_config.pkl", 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.